In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from collections import Counter

In [2]:
mainDF = pd.read_excel("Data//October_10//LabHoursData.xlsx")
mainDF.columns

Index(['#', 'Machine Type', 'Machine', 'Wing', 'Floor', 'Run No', 'Pcs',
       'StartDate', 'StartTime', 'StopDate', 'StopTime', 'Run %',
       'LateStartReason', 'Stop Reasons', 'Recipe', 'Run Intensity',
       'Ini Category', 'Grown Category', 'RunStatus', 'IniAvgHeight',
       'ProdHeight', 'GrownHeight', 'GrowingHours', 'BreakdownTime',
       'From To Hours', 'Actual Hours', 'RunHours', 'IniWeight', 'FinalWeight',
       'GrownWeight', 'Width', 'Length', 'Dmtr', 'Unit', 'Month', 'GrRate'],
      dtype='object')

In [3]:
#remove = "BreakdownTime", "GrowingHours"
final_DF = mainDF[["Machine Type", "Machine", "GrownWeight", "ProdHeight", "Dmtr", "GrowingHours",
                   "Actual Hours", "GrRate", "Stop Reasons", "LateStartReason"]]
#final_DF[final_DF.isna().any(axis=1)]
final_DF = final_DF.rename(columns = {'Machine Type':'Machine_Type', 'Stop Reasons':'Stop_Reasons', 'Actual Hours':'Actual_Hours'})

final_DF.tail(2)

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,GrowingHours,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
5303,New,AG42,59.228665,3.358553,41.13,174.98,192.0,12.14,"TIDE O , N/A",ON TIME
5304,NaN,NaN,NaN,NaN,NaN,"Sum: 658,339.71",NaN,Avg: 12.51,NaN,NaN


In [4]:
nullID = final_DF[final_DF[['Machine_Type','Machine','GrownWeight', 'ProdHeight','Dmtr', \
                            'Actual_Hours', 'GrRate']].isna().any(axis=1)].index
nullID

Int64Index([5304], dtype='int64')

In [5]:
final_DF = final_DF.drop(nullID)

In [6]:
final_DF.head(2)

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,GrowingHours,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
0,Old,CA2,97.87,3.777692,50.71,180,180.0,13.67,"TIDE O , V LEAK",ON TIME
1,Old,CA2,76.91,4.452222,46.92,160,160.0,13.56,"TIDE O , V LEAK",ON TIME


In [7]:
def count_words(df_temp,column_name):
    df_temp[column_name] = df_temp[column_name].str.rstrip('')
    df_temp[column_name] = df_temp[column_name].str.lstrip('')

    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(' ,',',')
    df_temp[column_name] = df_temp[column_name].str.replace(', ',',')
    df_temp[column_name] = df_temp[column_name].str.replace(',,',',')

    df_temp[column_name] = df_temp[column_name].str.replace('\' ','\'')
    df_temp[column_name] = df_temp[column_name].str.replace(' \'','\'')

    df_temp[column_name] = df_temp[column_name].str.rstrip(',')
    df_temp[column_name] = df_temp[column_name].str.lstrip(',')

    df_temp[column_name] = df_temp[column_name].str.split(',')
    df_temp[column_name] = df_temp[column_name].apply(lambda x: Counter(x))
    
    df_temp[column_name] = df_temp[column_name].apply(lambda x: dict(x))
    
    return df_temp

In [8]:
#join all string values of Stop_Reasons in groupby
df_Stop_reason = final_DF.groupby(['Machine','Machine_Type'])['Stop_Reasons'].agg(lambda x: ','.join(x.dropna())).reset_index()

#join all string values of LateStartReason in groupby
df_LateStartReason = final_DF.groupby(['Machine','Machine_Type'])['LateStartReason'].agg(lambda x: ','.join(x.dropna())).reset_index()

In [9]:
final_DF.columns

Index(['Machine_Type', 'Machine', 'GrownWeight', 'ProdHeight', 'Dmtr',
       'GrowingHours', 'Actual_Hours', 'GrRate', 'Stop_Reasons',
       'LateStartReason'],
      dtype='object')

In [10]:
df_Stop_reason['Stop_Reasons_counts'] = df_Stop_reason['Stop_Reasons']
df_Stop_reason = count_words(df_Stop_reason,'Stop_Reasons_counts')

df_LateStartReason['LateStartReason_counts'] = df_LateStartReason['LateStartReason']
df_LateStartReason = count_words(df_LateStartReason,'LateStartReason_counts')

In [11]:
final_DF.GrownWeight = pd.to_numeric(final_DF.GrownWeight)
final_DF.GrowingHours = pd.to_numeric(final_DF.GrowingHours)
final_DF.ProdHeight = pd.to_numeric(final_DF.ProdHeight)
final_DF.Dmtr = pd.to_numeric(final_DF.Dmtr)
final_DF.Actual_Hours = pd.to_numeric(final_DF.Actual_Hours)
final_DF.GrRate = pd.to_numeric(final_DF.GrRate)

In [12]:
df_machine_less35 = pd.merge(
        final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight','GrowingHours'].sum().reset_index(),
        final_DF.groupby(['Machine', 'Machine_Type'])['GrRate', 'Actual_Hours', 'ProdHeight','Dmtr'].mean().reset_index(), 
                                      how="left", on=["Machine", "Machine_Type"])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9072\1619885743.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine', 'Machine_Type'])['GrownWeight','GrowingHours'].sum().reset_index(),
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9072\1619885743.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  final_DF.groupby(['Machine', 'Machine_Type'])['GrRate', 'Actual_Hours', 'ProdHeight','Dmtr'].mean().reset_index(),


In [13]:
df_machine_less35.head(2)

,Machine,Machine_Type,GrownWeight,GrowingHours,GrRate,Actual_Hours,ProdHeight,Dmtr
0,AA1,Old,264.03016,675.31,12.855,136.533333,3.069045,41.901667
1,AA10,Old,272.45215,675.65,11.910,140.900000,3.240711,44.670000


In [14]:
# df_machine_less35.index.name = "ID"
# df_machine_less35.reset_index(inplace=True)

In [15]:
df_machine_less35.head(2)

,Machine,Machine_Type,GrownWeight,GrowingHours,GrRate,Actual_Hours,ProdHeight,Dmtr
0,AA1,Old,264.03016,675.31,12.855,136.533333,3.069045,41.901667
1,AA10,Old,272.45215,675.65,11.910,140.900000,3.240711,44.670000


In [16]:
df_machine_less35["GrownWeight_less_35"] = df_machine_less35['GrownWeight'] - (df_machine_less35["GrownWeight"] * 35 / 100)
df_machine_less35["GrRate_rank"] = df_machine_less35['GrRate'].apply(lambda x: 1 if (x>12.50 and x<=13.00 ) else 0)

In [ ]:
#=============================================================================

In [ ]:
#==============================================================================

In [17]:
def add_sorting_rank_to_column(temp_df, column_name, ascending_order):
    temp_df[column_name] = pd.to_numeric(temp_df[column_name])
    temp_df = temp_df.sort_values(by=[column_name], ascending=ascending_order).reset_index()
    temp_df.index.name = column_name + "_rank"
    temp_df = temp_df.reset_index()
    temp_df[column_name + "_rank"] += 1
    temp_df.index.name = "delete_" + column_name
    return temp_df

In [18]:
df_machine_less35

,Machine,Machine_Type,GrownWeight,GrowingHours,GrRate,Actual_Hours,ProdHeight,Dmtr,GrownWeight_less_35,GrRate_rank
0,AA1,Old,264.030160,675.31,12.855000,136.533333,3.069045,41.901667,171.619604,1
1,AA10,Old,272.452150,675.65,11.910000,140.900000,3.240711,44.670000,177.093898,0
2,AA11,Old,264.602140,658.81,11.496667,155.666667,3.573384,44.988333,171.991391,0
3,AA12,Old,244.256410,685.90,12.536000,156.850000,3.422741,42.404000,158.766667,1
4,AA13,Old,265.286690,687.14,11.864000,169.900000,3.780318,44.416000,172.436349,0
...,...,...,...,...,...,...,...,...,...,...
1023,CC16,Old,296.671520,713.64,13.554000,156.000000,3.928746,44.294000,192.836488,0
1024,CC17,Old,328.814920,697.57,13.698571,143.042857,3.257229,46.428571,213.729698,0
1025,CC20,Old,285.491085,696.82,13.895000,146.233333,3.314273,42.951667,185.569205,0
1026,CC21,Old,363.617610,705.16,14.710000,159.516667,3.147714,46.235000,236.351447,0


In [19]:
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "GrownWeight", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "GrowingHours", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "Actual_Hours", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "ProdHeight", False)
df_machine_less35 = add_sorting_rank_to_column(df_machine_less35, "Dmtr", False)
df_machine_less35 = df_machine_less35.reset_index()
df_machine_less35 = df_machine_less35.drop(["delete_Dmtr","delete_ProdHeight","delete_Actual_Hours","delete_GrownWeight","delete_GrowingHours"], axis=1)

In [20]:
df_machine_less35

,Dmtr_rank,ProdHeight_rank,Actual_Hours_rank,GrowingHours_rank,GrownWeight_rank,index,Machine,Machine_Type,GrownWeight,GrowingHours,GrRate,Actual_Hours,ProdHeight,Dmtr,GrownWeight_less_35,GrRate_rank
0,1,213,891,727,1,328,AE56,Old,553.110150,647.36,14.296000,131.506000,3.702460,62.748000,359.521597,0
1,2,344,181,381,3,298,AE29,Old,424.726115,688.06,13.332000,181.600000,3.398941,55.656000,276.071975,0
2,3,612,1001,818,4,265,AD40,Old,424.578360,606.50,16.151667,108.200000,2.821468,54.813333,275.975934,0
3,4,134,868,689,2,274,AD7,Old,452.762620,654.82,14.260000,133.566667,3.975168,53.981667,294.295703,0
4,5,265,225,301,6,169,AC23,Old,405.883690,692.40,13.482000,178.230000,3.567276,53.246000,263.824399,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,1024,532,274,85,602,339,AF15,Old,246.920125,705.15,13.750000,174.200000,3.005717,37.030000,160.498081,0
1024,1025,878,748,492,903,324,AE52,Old,198.579320,680.16,13.010000,143.285000,2.253355,36.573333,129.076558,0
1025,1026,683,1021,779,859,25,AA32,Old,211.677270,626.98,15.960000,91.375000,2.669381,35.678750,137.590225,0
1026,1027,506,996,701,799,148,AB74,Old,227.139720,653.00,14.780000,108.883333,3.059479,32.828333,147.640818,0


In [21]:
df_machine_less35['Total_rank'] = df_machine_less35['GrownWeight_rank'] + df_machine_less35['Dmtr_rank'] + \
    df_machine_less35['GrowingHours_rank'] + df_machine_less35['Actual_Hours_rank'] + df_machine_less35["ProdHeight_rank"]

In [22]:
df_results = pd.merge(pd.merge(df_machine_less35,df_Stop_reason,how="left", on=["Machine", "Machine_Type"]),df_LateStartReason,how="left", on=["Machine", "Machine_Type"])

In [23]:
df_results.columns

Index(['Dmtr_rank', 'ProdHeight_rank', 'Actual_Hours_rank',
       'GrowingHours_rank', 'GrownWeight_rank', 'index', 'Machine',
       'Machine_Type', 'GrownWeight', 'GrowingHours', 'GrRate', 'Actual_Hours',
       'ProdHeight', 'Dmtr', 'GrownWeight_less_35', 'GrRate_rank',
       'Total_rank', 'Stop_Reasons', 'Stop_Reasons_counts', 'LateStartReason',
       'LateStartReason_counts'],
      dtype='object')

In [24]:
df_results = df_results[['index','Machine_Type','Machine','GrownWeight','GrownWeight_less_35','Dmtr','Actual_Hours', 
                         'GrowingHours','ProdHeight','GrRate','GrownWeight_rank','Actual_Hours_rank', 'GrowingHours_rank',
                         'ProdHeight_rank','Dmtr_rank','GrRate_rank','Total_rank','Stop_Reasons_counts', 'LateStartReason_counts']]

In [25]:
df_results.head(2)

,index,Machine_Type,Machine,GrownWeight,GrownWeight_less_35,Dmtr,Actual_Hours,GrowingHours,ProdHeight,GrRate,GrownWeight_rank,Actual_Hours_rank,GrowingHours_rank,ProdHeight_rank,Dmtr_rank,GrRate_rank,Total_rank,Stop_Reasons_counts,LateStartReason_counts
0,328,Old,AE56,553.110150,359.521597,62.748,131.506,647.36,3.702460,14.296,1,891,727,213,1,0,1833,"{'TIDE O': 3, 'N/A': 5, 'PSSOC': 1, 'V LEAK': 1}","{'MW': 2, 'HLD CLN': 1, 'ARC': 1, 'W LINE': 1}"
1,298,Old,AE29,424.726115,276.071975,55.656,181.600,688.06,3.398941,13.332,3,181,381,344,2,0,911,"{'TIDE O': 5, 'V LEAK': 1, 'N/A': 4}","{'CHMB CNT': 1, 'CHMB CHNG': 1, 'HLD CLN': 2, ..."


In [26]:
df_results = df_results.sort_values(by=['index'])
df_results = df_results.set_index("index")
if df_results.index[0] == 0:
    df_results.index += 1

In [27]:
df_results

,Machine_Type,Machine,GrownWeight,GrownWeight_less_35,Dmtr,Actual_Hours,GrowingHours,ProdHeight,GrRate,GrownWeight_rank,Actual_Hours_rank,GrowingHours_rank,ProdHeight_rank,Dmtr_rank,GrRate_rank,Total_rank,Stop_Reasons_counts,LateStartReason_counts
index,,,,,,,,,,,,,,,,,,
1,Old,AA1,264.030160,171.619604,41.901667,136.533333,675.31,3.069045,12.855000,369,837,546,503,755,1,3010,"{'TIDE O': 4, 'V LEAK': 1, 'HEAT': 1, 'N/A': 5...","{'ON TIME': 4, 'GENERAL MAINTENANCE': 1, 'CHMB..."
2,Old,AA10,272.452150,177.093898,44.670000,140.900000,675.65,3.240711,11.910000,273,777,542,426,112,0,2130,"{'TIDE O': 4, 'N/A': 6, 'HEAT': 1, 'POWER TRIP...","{'CHMB CNT': 1, 'ON TIME': 4, 'GENERAL MAINTEN..."
3,Old,AA11,264.602140,171.991391,44.988333,155.666667,658.81,3.573384,11.496667,362,548,664,262,88,0,1924,"{'TIDE O': 4, 'N/A': 6, 'HEAT': 1, 'DLC BLB': 1}","{'ON TIME': 4, 'GENERAL MAINTENANCE': 1, 'POWE..."
4,Old,AA12,244.256410,158.766667,42.404000,156.850000,685.90,3.422741,12.536000,638,524,419,330,618,1,2529,"{'TIDE O': 4, 'N/A': 5, 'REF PWR ER': 1}","{'ON TIME': 3, 'GENERAL MAINTENANCE': 1, 'STG'..."
5,Old,AA13,265.286690,172.436349,44.416000,169.900000,687.14,3.780318,11.864000,356,329,396,185,148,0,1414,"{'TIDE O': 4, 'N/A': 5, 'HEAT': 1}","{'ON TIME': 4, 'GENERAL MAINTENANCE': 1}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,Old,CC16,296.671520,192.836488,44.294000,156.000000,713.64,3.928746,13.554000,77,542,23,145,167,0,954,"{'TIDE O': 4, 'N/A': 5, 'DLC BRK': 1}",{'ON TIME': 5}
1025,Old,CC17,328.814920,213.729698,46.428571,143.042857,697.57,3.257229,13.698571,30,752,215,418,22,0,1437,"{'TIDE O': 5, 'V LEAK': 1, 'N/A': 6, 'HEAT': 2}",{'ON TIME': 7}
1026,Old,CC20,285.491085,185.569205,42.951667,146.233333,696.82,3.314273,13.895000,144,702,229,386,479,0,1940,"{'TIDE O': 4, 'N/A': 6, 'HEAT': 1, 'WA': 1}","{'ON TIME': 5, 'MAINTENANCE': 1}"


In [28]:
if os.path.exists("final.xlsx"):
    os.remove("final.xlsx")
    
df_results.to_excel("final.xlsx", header=1, index_label="ID")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [29]:
m,t = 'AA24','Old'
final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)]

,Machine_Type,Machine,GrownWeight,ProdHeight,Dmtr,GrowingHours,Actual_Hours,GrRate,Stop_Reasons,LateStartReason
1177,Old,AA24,30.14113,2.104109,44.01,77.92,180.0,11.89,"TIDE O , N/A",ON TIME
1178,Old,AA24,31.06000,4.470000,44.72,84.08,84.5,11.66,"HEAT , N/A",ON TIME
1260,Old,AA24,75.32500,5.313000,46.68,201.17,201.1,10.04,"TIDE O , N/A",GENERAL MAINTENANCE
1261,Old,AA24,45.16500,4.861666,43.56,132.00,132.0,11.21,"TIDE O , N/A",ON TIME
1262,Old,AA24,66.96000,5.077272,44.29,181.00,181.0,10.66,"TIDE O , N/A",NaN


In [30]:
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrownWeight.sum())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrowingHours.sum())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Actual_Hours.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].ProdHeight.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].Dmtr.mean())
print(final_DF[(final_DF.Machine==m)&(final_DF.Machine_Type==t)].GrRate.mean())

248.65112999999997
676.17
155.72
4.3652093999999995
44.652
11.092000000000002
